In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

C:\Users\cudalab07\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [20]:
# Initializing parameters
epochs = 25
batch_size = 16

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\cudalab07\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


# Defining the Architecture of LSTM and hyperparameter tuning

In [21]:
# setting the hyperparameters
n_hidden = [16, 32, 48, 64]
dropouts = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

max_score = [0, 0]
for n_hid in n_hidden:
    for drop in dropouts:
# Initiliazing the sequential model
        model = Sequential()
# Configuring the parameters
        model.add(LSTM(n_hid, input_shape=(timesteps, input_dim), return_sequences = True))
        model.add(LSTM(n_hid, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
        model.add(Dropout(drop))
# Adding a dense output layer with sigmoid activation
        model.add(Dense(n_classes, activation='sigmoid'))
        # Compiling the model
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        # Training the model
        model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)
        score = model.evaluate(X_test, Y_test)
        if max_score[1] < score[1]:
            max_score = score
    print("-"*125)

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 64s 9ms/step - loss: 1.0863 - acc: 0.5147 - val_loss: 0.9392 - val_acc: 0.5684
Epoch 2/25
7352/7352 [==============================] - 64s 9ms/step - loss: 0.7385 - acc: 0.6610 - val_loss: 0.8253 - val_acc: 0.6593
Epoch 3/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.5820 - acc: 0.7376 - val_loss: 0.5524 - val_acc: 0.7401
Epoch 4/25
7352/7352 [==============================] - 59s 8ms/step - loss: 0.4476 - acc: 0.7976 - val_loss: 0.6006 - val_acc: 0.7855
Epoch 5/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.3455 - acc: 0.8784 - val_loss: 0.6971 - val_acc: 0.8012
Epoch 6/25
7352/7352 [==============================] - 59s 8ms/step - loss: 0.2176 - acc: 0.9185 - val_loss: 0.6032 - val_acc: 0.8504
Epoch 7/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1933 - acc: 0.9325 - val_loss: 0.7462 - val_acc: 0.8358
Epoch 8

7352/7352 [==============================] - 65s 9ms/step - loss: 0.1386 - acc: 0.9465 - val_loss: 0.3619 - val_acc: 0.9036
Epoch 10/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1355 - acc: 0.9510 - val_loss: 0.4621 - val_acc: 0.9026
Epoch 11/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1423 - acc: 0.9470 - val_loss: 0.4157 - val_acc: 0.9057
Epoch 12/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1407 - acc: 0.9475 - val_loss: 0.7177 - val_acc: 0.8449
Epoch 13/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1365 - acc: 0.9497 - val_loss: 0.5115 - val_acc: 0.9046
Epoch 14/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1295 - acc: 0.9505 - val_loss: 0.4113 - val_acc: 0.9026
Epoch 15/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1227 - acc: 0.9532 - val_loss: 0.3937 - val_acc: 0.9074
Epoch 16/25
7352/7352 [==============================] - 65s

Epoch 18/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1289 - acc: 0.9514 - val_loss: 0.4474 - val_acc: 0.9141
Epoch 19/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1227 - acc: 0.9494 - val_loss: 0.3396 - val_acc: 0.9104
Epoch 20/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1346 - acc: 0.9509 - val_loss: 0.3947 - val_acc: 0.8924
Epoch 21/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1422 - acc: 0.9479 - val_loss: 0.5559 - val_acc: 0.8918
Epoch 22/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1310 - acc: 0.9471 - val_loss: 0.4713 - val_acc: 0.9101
Epoch 23/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1314 - acc: 0.9527 - val_loss: 0.3811 - val_acc: 0.9084
Epoch 24/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1398 - acc: 0.9512 - val_loss: 0.3996 - val_acc: 0.9148
Epoch 25/25
2947/2947 [=========================

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 63s 9ms/step - loss: 1.1422 - acc: 0.5054 - val_loss: 0.9580 - val_acc: 0.5453
Epoch 2/25
7352/7352 [==============================] - 61s 8ms/step - loss: 0.7448 - acc: 0.6601 - val_loss: 0.7111 - val_acc: 0.6902
Epoch 3/25
7352/7352 [==============================] - 61s 8ms/step - loss: 0.5763 - acc: 0.7647 - val_loss: 0.5699 - val_acc: 0.7822
Epoch 4/25
7352/7352 [==============================] - 66s 9ms/step - loss: 0.3861 - acc: 0.8712 - val_loss: 0.4896 - val_acc: 0.8517
Epoch 5/25
7352/7352 [==============================] - 66s 9ms/step - loss: 0.2567 - acc: 0.9128 - val_loss: 0.3388 - val_acc: 0.8887
Epoch 6/25
7352/7352 [==============================] - 66s 9ms/step - loss: 0.1905 - acc: 0.9313 - val_loss: 0.2593 - val_acc: 0.9135
Epoch 7/25
7352/7352 [==============================] - 66s 9ms/step - loss: 0.1783 - acc: 0.9403 - val_loss: 0.3437 - val_acc: 0.8958
Epoch 8

7352/7352 [==============================] - 60s 8ms/step - loss: 0.1441 - acc: 0.9478 - val_loss: 0.4298 - val_acc: 0.9057
Epoch 10/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1485 - acc: 0.9436 - val_loss: 0.3389 - val_acc: 0.9053
Epoch 11/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1379 - acc: 0.9444 - val_loss: 0.5011 - val_acc: 0.8761
Epoch 12/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1385 - acc: 0.9484 - val_loss: 0.3915 - val_acc: 0.9087
Epoch 13/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1482 - acc: 0.9449 - val_loss: 0.4220 - val_acc: 0.8999
Epoch 14/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1450 - acc: 0.9476 - val_loss: 0.5056 - val_acc: 0.8904
Epoch 15/25
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1397 - acc: 0.9474 - val_loss: 0.4507 - val_acc: 0.8972
Epoch 16/25
7352/7352 [==============================] - 60s

...

7352/7352 [==============================] - 62s 8ms/step - loss: 0.1588 - acc: 0.9426 - val_loss: 0.3942 - val_acc: 0.8955
Epoch 10/25
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1493 - acc: 0.9441 - val_loss: 0.3682 - val_acc: 0.9002
Epoch 11/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1609 - acc: 0.9426 - val_loss: 0.4761 - val_acc: 0.9030
Epoch 12/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1425 - acc: 0.9484 - val_loss: 0.3624 - val_acc: 0.9141
Epoch 13/25
7352/7352 [==============================] - 63s 9ms/step - loss: 0.1271 - acc: 0.9504 - val_loss: 0.4918 - val_acc: 0.8884
Epoch 14/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1359 - acc: 0.9498 - val_loss: 0.3272 - val_acc: 0.9121
Epoch 15/25
7352/7352 [==============================] - 62s 8ms/step - loss: 0.1324 - acc: 0.9501 - val_loss: 0.4709 - val_acc: 0.8962
Epoch 16/25
7352/7352 [==============================] - 62s

Epoch 18/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1281 - acc: 0.9520 - val_loss: 0.4246 - val_acc: 0.9111
Epoch 19/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1274 - acc: 0.9499 - val_loss: 0.3344 - val_acc: 0.9189
Epoch 20/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1214 - acc: 0.9536 - val_loss: 0.4344 - val_acc: 0.9101
Epoch 21/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1183 - acc: 0.9532 - val_loss: 0.4133 - val_acc: 0.9077
Epoch 22/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1208 - acc: 0.9532 - val_loss: 0.3446 - val_acc: 0.9128
Epoch 23/25
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1370 - acc: 0.9455 - val_loss: 0.4410 - val_acc: 0.9138
Epoch 24/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1287 - acc: 0.9524 - val_loss: 0.3927 - val_acc: 0.9114
Epoch 25/25
2947/2947 [=========================

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 68s 9ms/step - loss: 1.0506 - acc: 0.5394 - val_loss: 0.8444 - val_acc: 0.5986
Epoch 2/25
7352/7352 [==============================] - 70s 10ms/step - loss: 0.6876 - acc: 0.6914 - val_loss: 0.6221 - val_acc: 0.7713
Epoch 3/25
7352/7352 [==============================] - 70s 9ms/step - loss: 0.4189 - acc: 0.8500 - val_loss: 0.5290 - val_acc: 0.8398
Epoch 4/25
7352/7352 [==============================] - 69s 9ms/step - loss: 0.2867 - acc: 0.9008 - val_loss: 0.3840 - val_acc: 0.8867
Epoch 5/25
7352/7352 [==============================] - 70s 9ms/step - loss: 0.2110 - acc: 0.9295 - val_loss: 0.4112 - val_acc: 0.8890
Epoch 6/25
7352/7352 [==============================] - 70s 9ms/step - loss: 0.1837 - acc: 0.9344 - val_loss: 0.3832 - val_acc: 0.8907
Epoch 7/25
7352/7352 [==============================] - 69s 9ms/step - loss: 0.1639 - acc: 0.9393 - val_loss: 0.3742 - val_acc: 0.8836
Epoch 

7352/7352 [==============================] - 65s 9ms/step - loss: 0.1466 - acc: 0.9480 - val_loss: 0.5103 - val_acc: 0.8772
Epoch 10/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1415 - acc: 0.9493 - val_loss: 0.3776 - val_acc: 0.8928
Epoch 11/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1393 - acc: 0.9493 - val_loss: 0.3122 - val_acc: 0.9152
Epoch 12/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1315 - acc: 0.9497 - val_loss: 0.3947 - val_acc: 0.9057
Epoch 13/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1347 - acc: 0.9484 - val_loss: 0.4121 - val_acc: 0.8907
Epoch 14/25
7352/7352 [==============================] - 72s 10ms/step - loss: 0.1419 - acc: 0.9502 - val_loss: 0.2839 - val_acc: 0.9162
Epoch 15/25
7352/7352 [==============================] - 71s 10ms/step - loss: 0.1330 - acc: 0.9528 - val_loss: 0.4156 - val_acc: 0.8996
Epoch 16/25
7352/7352 [==============================] - 6

Epoch 18/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1161 - acc: 0.9490 - val_loss: 0.2874 - val_acc: 0.9196
Epoch 19/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1186 - acc: 0.9520 - val_loss: 0.3468 - val_acc: 0.9172
Epoch 20/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1307 - acc: 0.9509 - val_loss: 0.3451 - val_acc: 0.9209
Epoch 21/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1266 - acc: 0.9489 - val_loss: 0.3016 - val_acc: 0.9243
Epoch 22/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1192 - acc: 0.9510 - val_loss: 0.3248 - val_acc: 0.9230
Epoch 23/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1195 - acc: 0.9535 - val_loss: 0.3952 - val_acc: 0.9128
Epoch 24/25
7352/7352 [==============================] - 65s 9ms/step - loss: 0.1206 - acc: 0.9509 - val_loss: 0.5859 - val_acc: 0.8921
Epoch 25/25
2947/2947 [=========================

# Results and Conclusion

In [29]:
print("The model gives test accuracy of almost {}% with the help of hyperparameter tuning".format(round(max_score[1], 2) * 100))

The model gives test accuracy of almost 93.0% with the help of hyperparameter tuning


In [27]:
# Training based on the chosen hyperparameter values
# As observed from above we take 64 number of LSTM units and 0.3 as the dropout rate
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences = True)) 
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.3))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_68 (LSTM)               (None, 128, 64)           18944     
_________________________________________________________________
lstm_69 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_34 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 6)                 390       
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)
# Testing on  test data
score = model.evaluate(X_test, Y_test)

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 75s 10ms/step - loss: 1.0492 - acc: 0.5529 - val_loss: 0.8673 - val_acc: 0.5724
Epoch 2/25
7352/7352 [==============================] - 68s 9ms/step - loss: 0.7265 - acc: 0.6919 - val_loss: 0.6137 - val_acc: 0.7479
Epoch 3/25
7352/7352 [==============================] - 68s 9ms/step - loss: 0.5473 - acc: 0.7554 - val_loss: 0.5462 - val_acc: 0.7760
Epoch 4/25
7352/7352 [==============================] - 68s 9ms/step - loss: 0.3571 - acc: 0.8630 - val_loss: 0.4616 - val_acc: 0.8582
Epoch 5/25
7352/7352 [==============================] - 68s 9ms/step - loss: 0.2134 - acc: 0.9256 - val_loss: 0.2990 - val_acc: 0.8914
Epoch 6/25
7352/7352 [==============================] - 68s 9ms/step - loss: 0.1768 - acc: 0.9365 - val_loss: 0.3593 - val_acc: 0.8806
Epoch 7/25
7352/7352 [==============================] - 68s 9ms/step - loss: 0.1748 - acc: 0.9388 - val_loss: 0.2433 - val_acc: 0.9104
Epoch 

In [30]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        25        0                   0   
SITTING                  2      414        73        0                   0   
STANDING                 0       89       442        1                   0   
WALKING                  0        0         0      487                   4   
WALKING_DOWNSTAIRS       0        1         0        1                 411   
WALKING_UPSTAIRS         0        1         2       10                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             2  
SITTING                            2  
STANDING                           0  
WALKING                            5  
WALKING_DOWNSTAIRS                 7  
WALKING_UPSTAIRS                 458  


In [31]:
score

[0.30954647953625675, 0.9236511706820495]

- With a simple 3 layers (2 layers of LSTM included) architecture we got almost 93% accuracy after 24 epochs and a loss of almost 0.30